# coco format to yolo5 format

In [1]:
import json
import os
import numpy as np
from tqdm import tqdm
import shutil
import cv2

In [2]:
def get_img_shape(path):
    img = cv2.imread(path)
    try:
        return img.shape
    except AttributeError:
        print('error!', path)
        return (None, None, None)
    
def convert_labels(path, x1, y1, x2, y2):
    """
    Definition: Parses label files to extract label and bounding box
    coordinates. Converts (x1, y1, x1, y2) KITTI format to
    (x, y, width, height) normalized YOLO format.
    """
    def sorting(l1, l2):
        if l1 > l2:
            lmax, lmin = l1, l2
            return lmax, lmin
        else:
            lmax, lmin = l2, l1
            return lmax, lmin
    size = get_img_shape(path)
    xmax, xmin = sorting(x1, x2)
    ymax, ymin = sorting(y1, y2)
    dw = 1./size[1]
    dh = 1./size[0]
    x = (xmin + xmax)/2.0
    y = (ymin + ymax)/2.0
    w = xmax - xmin
    h = ymax - ymin
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def generate_label(data, img_root, store_root):
    check_set = set()
    for i in tqdm(range(len(data['annotations']))):
        image_id = str(data['annotations'][i]['image_id'])
        category_id = str(data['annotations'][i]['category_id'])
        bbox = data['annotations'][i]['bbox']
        
        image_name = '0'*(12-len(image_id)) + image_id + ".jpg"
        image_path = os.path.join(img_root, image_name)
        kitti_bbox = [bbox[0], bbox[1], bbox[2] + bbox[0], bbox[3] + bbox[1]]
        yolo_bbox = convert_labels(image_path, kitti_bbox[0], kitti_bbox[1], kitti_bbox[2], kitti_bbox[3])
        filename = '0'*(12-len(image_id)) + image_id + ".txt"
        store_path = os.path.join(store_root, filename)
        content = category_id + " " + str(yolo_bbox[0]) + " " + str(yolo_bbox[1]) + " " + str(yolo_bbox[2]) + " " + str(yolo_bbox[3])
        if image_id in check_set:
            # Append to file files
            file = open(store_path, "a")
            file.write("\n")
            file.write(content)
            file.close()
        elif image_id not in check_set:
            check_set.add(image_id)
            # Write files
            file = open(store_path, "w")
            file.write(content)
            file.close()

In [3]:
train_anno_path = '/mnt/data/coco/labels/annotations/instances_train2017.json'
val_anno_path = '/mnt/data/coco/labels/annotations/instances_val2017.json'
with open(val_anno_path) as f:
    val_ann_data = json.loads(f.read())
    
with open(train_anno_path) as f:
    train_ann_data = json.loads(f.read())

In [4]:
generate_label(val_ann_data, 
               '/mnt/data/coco/images/val2017/', 
               '/mnt/data/coco/labels/val2017/')

100%|██████████| 36781/36781 [03:17<00:00, 185.84it/s]


In [ ]:
generate_label(train_ann_data, '/mnt/data/coco/images/train2017/', '/mnt/data/coco/labels/train2017/')

 31%|███       | 263594/860001 [23:49<53:42, 185.10it/s]  